# Load Data

In [16]:
!pip install deepctr_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


/kaggle/input/clickthrough-rate-prediction/ad_10000records.csv
/kaggle/input/clickthrough-rate-prediction/ad.csv
/kaggle/input/clickthrough-rate-prediction/new_ad.csv
/kaggle/input/clickthrough-rate-prediction/ad (1).csv


In [2]:

nRowsRead = None # specify 'None' if want to read whole file
# Apply_Rate_2019.csv has 1200890 rows in reality, but we are only loading/previewing the first 1000 rows
data = pd.read_csv('/kaggle/input/clickthrough-rate-prediction/ad.csv', delimiter=',', nrows = nRowsRead)
data.dataframeName = 'Apply_Rate_2019.csv'
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 50000 rows and 10 columns


In [3]:
data.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad
0,78.84,30.0,68448.94,161.79,Front-line even-keeled website,Silvaton,Male,Peru,2016-04-04 03:57:48,1
1,60.23,35.0,66815.54,216.24,Front-line fresh-thinking open system,West Jeremyside,Male,Papua New Guinea,2016-06-18 16:02:34,0
2,46.20,49.0,62109.80,135.24,Enhanced maximized access,Lake Vanessa,Male,Chile,2016-06-26 07:01:47,1
3,87.85,31.0,50055.33,134.46,Total zero administration software,Port Sherrystad,Male,French Polynesia,2016-04-18 21:07:28,1
4,64.38,37.0,55121.65,232.54,Devolved regional moderator,South Patrickfort,Female,Bosnia and Herzegovina,2016-07-18 18:33:05,0


In [4]:
data.columns

Index(['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Ad Topic Line', 'City', 'Gender', 'Country',
       'Timestamp', 'Clicked on Ad'],
      dtype='object')

In [6]:
data=data[['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Ad Topic Line', 'City', 'Gender', 
        'Clicked on Ad']]

In [7]:
data.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Clicked on Ad
0,78.84,30.0,68448.94,161.79,Front-line even-keeled website,Silvaton,Male,1
1,60.23,35.0,66815.54,216.24,Front-line fresh-thinking open system,West Jeremyside,Male,0
2,46.20,49.0,62109.80,135.24,Enhanced maximized access,Lake Vanessa,Male,1
3,87.85,31.0,50055.33,134.46,Total zero administration software,Port Sherrystad,Male,1
4,64.38,37.0,55121.65,232.54,Devolved regional moderator,South Patrickfort,Female,0


# Define Columns

In [5]:
label='Clicked on Ad'

In [8]:
cat_features=['Ad Topic Line','City','Gender']

In [9]:
continue_var=[ 'Daily Time Spent on Site', 'Age', 'Area Income','Daily Internet Usage']

# Pre-processing

In [10]:
fill_mean = lambda x: x.fillna(x.mean())

for col in continue_var:
    data[col] = data[col].fillna(data[col].mean())
    data[col] = data[col].astype('float64')
data = data.fillna('-1', )


In [11]:
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 1))
data[continue_var] = mms.fit_transform(data[continue_var])


In [12]:
train=data

In [13]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c].astype(str))
    train[c] = le.transform(train[c].astype(str))

In [17]:

import math
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV


from deepctr_torch.inputs import SparseFeat, get_feature_names,DenseFeat
from deepctr_torch.models import DeepFM
import torch

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=train[feat].max() + 1, embedding_dim=30)
                              for feat in cat_features] + [DenseFeat(feat, 1, )
                                                              for feat in continue_var]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
        linear_feature_columns + dnn_feature_columns)
feature_names


train, test = train_test_split(train, test_size=0.2)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
train


device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

# DEEPFM

In [25]:


target=label


model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,256),  l2_reg_linear=0.1,
              dnn_use_bn=True,device=device,dnn_activation='relu', 
             l2_reg_dnn=0.2, l2_reg_embedding=1e-2, dnn_dropout=0.2)
model.compile("adam", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[label].values, batch_size=128, epochs=10, verbose=2
                    )

from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

cpu
Train on 40000 samples, validate on 0 samples, 313 steps per epoch
Epoch 1/10
3s - loss:  0.3136 - binary_crossentropy:  0.3035 - auc:  0.9438
Epoch 2/10
3s - loss:  0.2748 - binary_crossentropy:  0.2627 - auc:  0.9593
Epoch 3/10
3s - loss:  0.2691 - binary_crossentropy:  0.2548 - auc:  0.9621
Epoch 4/10
3s - loss:  0.2657 - binary_crossentropy:  0.2494 - auc:  0.9637
Epoch 5/10
3s - loss:  0.2643 - binary_crossentropy:  0.2460 - auc:  0.9648
Epoch 6/10
3s - loss:  0.2638 - binary_crossentropy:  0.2438 - auc:  0.9656
Epoch 7/10
4s - loss:  0.2623 - binary_crossentropy:  0.2407 - auc:  0.9662
Epoch 8/10
3s - loss:  0.2587 - binary_crossentropy:  0.2364 - auc:  0.9671
Epoch 9/10
3s - loss:  0.2610 - binary_crossentropy:  0.2367 - auc:  0.9676
Epoch 10/10
3s - loss:  0.2586 - binary_crossentropy:  0.2337 - auc:  0.9682

test LogLoss 0.2906
test roc_auc_score 0.8813
test f1 score 0.8822
test precision score 0.885
test recall score 0.8795
test accuracy score 0.8813


# xDeepFM

In [24]:


from deepctr_torch.models import xDeepFM
model =  xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,256),  l2_reg_linear=0.1,
              dnn_use_bn=True,device=device,dnn_activation='relu', 
             l2_reg_dnn=0.2, l2_reg_embedding=1e-2, dnn_dropout=0.2)
model.compile("adam", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[label].values, batch_size=128, epochs=8, verbose=2
                      )

from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

cpu
Train on 40000 samples, validate on 0 samples, 313 steps per epoch
Epoch 1/8
14s - loss:  0.3130 - binary_crossentropy:  0.3040
Epoch 2/8
14s - loss:  0.2705 - binary_crossentropy:  0.2613
Epoch 3/8
14s - loss:  0.2626 - binary_crossentropy:  0.2511
Epoch 4/8
14s - loss:  0.2586 - binary_crossentropy:  0.2452
Epoch 5/8
14s - loss:  0.2565 - binary_crossentropy:  0.2418
Epoch 6/8
14s - loss:  0.2548 - binary_crossentropy:  0.2392
Epoch 7/8
13s - loss:  0.2517 - binary_crossentropy:  0.2344
Epoch 8/8
14s - loss:  0.2508 - binary_crossentropy:  0.2323

test LogLoss 0.2896
test roc_auc_score 0.879
test f1 score 0.8762
test precision score 0.9053
test recall score 0.8489
test accuracy score 0.8787


# Wide & Deep

In [26]:


from deepctr_torch.models import WDL
model =  WDL(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,256),  l2_reg_linear=0.1,
                   dnn_use_bn=True,device=device,dnn_activation='relu', 
             l2_reg_dnn=0, l2_reg_embedding=1e-2, dnn_dropout=0)
model.compile("adam", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auroc"], )
history = model.fit(train_model_input, train[target].values, batch_size=128, epochs=8, verbose=2,
                )

from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

cpu
Train on 40000 samples, validate on 0 samples, 313 steps per epoch
Epoch 1/8
2s - loss:  0.3084 - binary_crossentropy:  0.3080
Epoch 2/8
3s - loss:  0.2638 - binary_crossentropy:  0.2626
Epoch 3/8
2s - loss:  0.2525 - binary_crossentropy:  0.2509
Epoch 4/8
2s - loss:  0.2455 - binary_crossentropy:  0.2433
Epoch 5/8
3s - loss:  0.2377 - binary_crossentropy:  0.2352
Epoch 6/8
3s - loss:  0.2305 - binary_crossentropy:  0.2275
Epoch 7/8
3s - loss:  0.2265 - binary_crossentropy:  0.2233
Epoch 8/8
2s - loss:  0.2217 - binary_crossentropy:  0.2180

test LogLoss 0.2958
test roc_auc_score 0.8819
test f1 score 0.8834
test precision score 0.8816
test recall score 0.8853
test accuracy score 0.8819


#  Improved Deep Cross 

In [27]:


from deepctr_torch.models import DCNMix
model =  DCNMix(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,256),  l2_reg_linear=0.1, dnn_use_bn=True,device=device,
             dnn_activation='relu', 
             l2_reg_dnn=0, l2_reg_embedding=1e-2, dnn_dropout=0)
model.compile("adam", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=128, epochs=8, verbose=2,
                       )

from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

cpu
Train on 40000 samples, validate on 0 samples, 313 steps per epoch
Epoch 1/8
6s - loss:  0.3046 - binary_crossentropy:  0.3040 - auc:  0.9432
Epoch 2/8
6s - loss:  0.2655 - binary_crossentropy:  0.2640 - auc:  0.9590
Epoch 3/8
6s - loss:  0.2550 - binary_crossentropy:  0.2531 - auc:  0.9617
Epoch 4/8
7s - loss:  0.2465 - binary_crossentropy:  0.2440 - auc:  0.9644
Epoch 5/8
6s - loss:  0.2399 - binary_crossentropy:  0.2366 - auc:  0.9663
Epoch 6/8
7s - loss:  0.2354 - binary_crossentropy:  0.2318 - auc:  0.9677
Epoch 7/8
6s - loss:  0.2310 - binary_crossentropy:  0.2269 - auc:  0.9692
Epoch 8/8
6s - loss:  0.2262 - binary_crossentropy:  0.2217 - auc:  0.9707

test LogLoss 0.2907
test roc_auc_score 0.8811
test f1 score 0.8843
test precision score 0.8719
test recall score 0.8969
test accuracy score 0.8813
